Consider the manufacturing of
pistons where each piston consists of a rod and a tube that need to be assembled together to form the piston. Each rod consists of the main body and a special kit that is welded to the rod (the kit needs to be collected from warehouse and then assembled). The rod body is sawn from a large metal stick. The tube can also be sawn from a larger tube. Both rod body and tube must be collected together from the warehouse to ensure that their diameters fit. If the tube is not available, it can be bought from an external supplier. In any case some welding is necessary to be done on the tube before it can be assembled with the rod. Finally, between
sawing and welding, both rod and tube must be cleared of metal cuts produced by sawing. Assume that welding and
sawing operations require ten time units, assembly operation requires five time units, clearing can be done in two time units, and the material is collected from
warehouse in one time unit. If the tube is bought from an external supplier then it takes fifty time units to get it. Moreover, tube and rod must cool-down after welding which takes five time units. We assume that shipPiston MaxTime = 70.

source: Barták, Roman, and Ondrej Cepek. "Temporal Networks with Alternatives: Complexity and Model." FLAIRS Conference. 2007.

![](../pics/2023-08-06-14-11-22.png)

In [1]:
from docplex.cp.model import CpoModel
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)


import matplotlib.pyplot as plt
%matplotlib inline
#Change the plot size
from pylab import rcParams
rcParams['figure.figsize'] = 35, 10
import docplex.cp.utils_visu as visu

In [12]:
Required_jobs = [['CollectKit',1, 'collect special kit from warehouse'],
                 ['AssembleKit', 5, 'assemble special kit' ],
                ['WeldRod',10 + 5, 'weld special kit to the rod, and cool-down'], 
                ['CollectMaterial',1, 'collect material for rod body'],
                ['SawRod',10+ 5, 'saw the rod body from a large metal stick'],
                ['WeldTube',10, 'necessary welding of tube, and cool-down'],  
                ['ClearRod', 2 ,  'rod must be cleared of metal cuts produced by sawing'  ] ,
                ['AssemblePiston',5 ,  'Assemble rod and tube'  ]]





Optional_jobs = [['SawTube',10, 'saw tube from a larger tube' ],
                 ['BuyTube',50, 'buy tube from an external supplier' ],
                ['ClearTube', 2 ,  'Tube must be cleared of metal cuts produced by sawing'  ] 
                ]
                 
                 
    
Required_jobs = pd.DataFrame( Required_jobs,  columns = ['job', 'duration', 'comments'])
Optional_jobs = pd.DataFrame( Optional_jobs,  columns = ['job', 'duration', 'comments'])

display(Required_jobs)
display(Optional_jobs)

Precedence = pd.DataFrame([['CollectKit', 'AssembleKit'],
                          ['AssembleKit', 'WeldRod'],
                          ['CollectMaterial','SawRod'],
                          ['SawRod','ClearRod'],
                          ['ClearRod','WeldRod'],
                          #['SawTube','ClearTube'],
                          ['AssembleKit', 'WeldRod'],
                          ['WeldTube', 'AssemblePiston'],
                          ['WeldRod',  'AssemblePiston'],
                          ['AssemblePiston', 'ShipPiston']]
                          ,columns = ["beforeTask", "afterTask"])

display(Precedence)

#Simultaneous_jobs= pd.DataFrame([

,job,duration,comments
0,CollectKit,1,collect special kit from warehouse
1,AssembleKit,5,assemble special kit
2,WeldRod,15,"weld special kit to the rod, and cool-down"
3,CollectMaterial,1,collect material for rod body
4,SawRod,15,saw the rod body from a large metal stick
5,WeldTube,10,"necessary welding of tube, and cool-down"
6,ClearRod,2,rod must be cleared of metal cuts produced by sawing
7,AssemblePiston,5,Assemble rod and tube


,job,duration,comments
0,SawTube,10,saw tube from a larger tube
1,BuyTube,50,buy tube from an external supplier
2,ClearTube,2,Tube must be cleared of metal cuts produced by sawing


,beforeTask,afterTask
0,CollectKit,AssembleKit
1,AssembleKit,WeldRod
2,CollectMaterial,SawRod
3,SawRod,ClearRod
4,ClearRod,WeldRod
5,AssembleKit,WeldRod
6,WeldTube,AssemblePiston
7,WeldRod,AssemblePiston
8,AssemblePiston,ShipPiston


In [13]:
mdl = CpoModel()

Required_jobs['Required_DV']=0
for index, row in Required_jobs.iterrows():
    dv =  mdl.interval_var(start = 0 , end = 1000, size=row.duration, name="{}".format(row.job))
    Required_jobs.at[index, 'Required_DV']=dv

display(Required_jobs.head())


# Optional_jobs['Optional_DV']=0
# for index, row in Optional_jobs.iterrows():
#     dv =  mdl.interval_var(start = 0 , end = 1000, optional = True , size=row.duration, name="{}".format(row.job))
#     Optional_jobs.at[index, 'Optional_DV']=dv


# display(Optional_jobs.head())

# ShipPiston = mdl.interval_var( start =0 , end = 70, name='ShipPiston')

,job,duration,comments,Required_DV
0,CollectKit,1,collect special kit from warehouse,"CollectKit = intervalVar(start=0, end=1000, size=1)"
1,AssembleKit,5,assemble special kit,"AssembleKit = intervalVar(start=0, end=1000, size=5)"
2,WeldRod,15,"weld special kit to the rod, and cool-down","WeldRod = intervalVar(start=0, end=1000, size=15)"
3,CollectMaterial,1,collect material for rod body,"CollectMaterial = intervalVar(start=0, end=1000, size=1)"
4,SawRod,15,saw the rod body from a large metal stick,"SawRod = intervalVar(start=0, end=1000, size=15)"


In [15]:
DV_dict= dict(zip(Required_jobs['job'], Required_jobs['Required_DV']))
#d2= dict(zip(Optional_jobs['job'], Optional_jobs['Optional_DV']))
#DV_dict.update(d2)
DV_dict['ShipPiston'] = ShipPiston
DV_dict

{'CollectKit': <docplex.cp.expression.CpoIntervalVar at 0x20e94652ff0>,
 'AssembleKit': <docplex.cp.expression.CpoIntervalVar at 0x20e94652f80>,
 'WeldRod': <docplex.cp.expression.CpoIntervalVar at 0x20e946533e0>,
 'CollectMaterial': <docplex.cp.expression.CpoIntervalVar at 0x20e94652570>,
 'SawRod': <docplex.cp.expression.CpoIntervalVar at 0x20e946527a0>,
 'WeldTube': <docplex.cp.expression.CpoIntervalVar at 0x20e946500b0>,
 'ClearRod': <docplex.cp.expression.CpoIntervalVar at 0x20e94653ed0>,
 'AssemblePiston': <docplex.cp.expression.CpoIntervalVar at 0x20e94653680>,
 'ShipPiston': <docplex.cp.expression.CpoIntervalVar at 0x20e94583d80>}

In [16]:
# Temporal constraints.TASK_PRECEDENCES  

Precedence ["beforeTaskDV"] = Precedence ["beforeTask"].apply(lambda x:DV_dict[x])
Precedence ["afterTaskDV"] = Precedence ["afterTask"].apply(lambda x:DV_dict[x])

display(Precedence)

for index, row in Precedence.iloc[0:1].iterrows():
    mdl.add( mdl.end_before_start(row.beforeTaskDV, row.afterTaskDV))

,beforeTask,afterTask,beforeTaskDV,afterTaskDV
0,CollectKit,AssembleKit,"CollectKit = intervalVar(start=0, end=1000, size=1)","AssembleKit = intervalVar(start=0, end=1000, size=5)"
1,AssembleKit,WeldRod,"AssembleKit = intervalVar(start=0, end=1000, size=5)","WeldRod = intervalVar(start=0, end=1000, size=15)"
2,CollectMaterial,SawRod,"CollectMaterial = intervalVar(start=0, end=1000, size=1)","SawRod = intervalVar(start=0, end=1000, size=15)"
3,SawRod,ClearRod,"SawRod = intervalVar(start=0, end=1000, size=15)","ClearRod = intervalVar(start=0, end=1000, size=2)"
4,ClearRod,WeldRod,"ClearRod = intervalVar(start=0, end=1000, size=2)","WeldRod = intervalVar(start=0, end=1000, size=15)"
5,AssembleKit,WeldRod,"AssembleKit = intervalVar(start=0, end=1000, size=5)","WeldRod = intervalVar(start=0, end=1000, size=15)"
6,WeldTube,AssemblePiston,"WeldTube = intervalVar(start=0, end=1000, size=10)","AssemblePiston = intervalVar(start=0, end=1000, size=5)"
7,WeldRod,AssemblePiston,"WeldRod = intervalVar(start=0, end=1000, size=15)","AssemblePiston = intervalVar(start=0, end=1000, size=5)"
8,AssemblePiston,ShipPiston,"AssemblePiston = intervalVar(start=0, end=1000, size=5)","ShipPiston = intervalVar(start=0, end=70)"


In [17]:
print("\nSolving model....")
msol = mdl.solve(TimeLimit=120,  trace_log=False)

# Print solution
print("Solve status: " + msol.get_solve_status())

msol.print_solution()



Solving model....
Solve status: Infeasible
-------------------------------------------------------------------------------
Model constraints: 1, variables: integer: 0, interval: 2, sequence: 0
Solve status: Infeasible
Search status: SearchCompleted, stop cause: SearchHasNotBeenStopped
Solve time: 0.01 sec
-------------------------------------------------------------------------------
No solution data


In [18]:
mdl.    

In [15]:
import os
import subprocess
CPLEX_path  = "C:/IBM/ILOG/CPLEX_Studio2211/opl/bin/x64_win64"
os.chdir(CPLEX_path)

# The path of the project that containts all the .project, OPL. and etc. files

path = "C:/Users/m_gor/Desktop/OptimizationDemos/CP/JobShop/"
mod_file =path+ "fact.mod"

command = "oplrun  %s " %(mod_file )
print(command)

output = subprocess.getoutput(command)
print(output)

oplrun  C:/Users/m_gor/Desktop/OptimizationDemos/CP/JobShop/fact.mod 

<<< setup


<<< generate

 ! --------------------------------------------------- CP Optimizer 22.1.1.0 --
 ! Satisfiability problem - 14 variables, 14 constraints
 ! Initial process time : 0.01s (0.01s extraction + 0.00s propagation)
 !  . Log search space  : 51.4 (before), 51.4 (after)
 !  . Memory usage      : 303.5 kB (before), 303.5 kB (after)
 ! Using parallel search with 8 workers.
 ! ----------------------------------------------------------------------------
 !               Branches  Non-fixed    W       Branch decision
 ! Using iterative diving.
 *                     11  0.02s        1            -
 ! ----------------------------------------------------------------------------
 ! Search completed, 1 solution found.
 ! ----------------------------------------------------------------------------
 ! Number of branches     : 96
 ! Number of fails        : 4
 ! Total memory usage     : 3.0 MB (2.6 MB CP Optimi